In [2]:
from transformers import AutoTokenizer, CLIPTextModel
import torch
from torch.nn import CosineSimilarity
from loguru import logger

/root/anaconda3/envs/lisa/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-15 11:28:10,826] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
cossim = CosineSimilarity(dim=0, eps=1e-6)

def calc_similarity(v1, v2):
    return cossim(v1, v2)

In [5]:

model_name_or_path = "/opt/product/LLaVA/checkpoints/clip-vit-large-patch14"
# model_name_or_path = "openai/clip-vit-large-patch14"
model = CLIPTextModel.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


In [6]:

prompts = [
        "panoptic segment the women",
        "segment a",
        "segment the",
        "segment all the object",
        "segment everything",
        "panoptic segment",
]
inputs = tokenizer(
        prompts,
        padding=True,
        return_tensors="pt",
)

 

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
flattened_hidden_state = torch.flatten(last_hidden_state, 1, -1)
for i1, label1 in enumerate(prompts):
    for i2, label2 in enumerate(prompts):
        if i2 >= i1:
            print(
                    f"{label1} <-> {label2} = {calc_similarity(flattened_hidden_state[i1], flattened_hidden_state[i2]):.4f}"
                )

panoptic segment the women <-> panoptic segment the women = 1.0000
panoptic segment the women <-> segment a = 0.5076
panoptic segment the women <-> segment the = 0.4969
panoptic segment the women <-> segment all the object = 0.4840
panoptic segment the women <-> segment everything = 0.4883
panoptic segment the women <-> panoptic segment = 0.8298
segment a <-> segment a = 1.0000
segment a <-> segment the = 0.8692
segment a <-> segment all the object = 0.7368
segment a <-> segment everything = 0.7949
segment a <-> panoptic segment = 0.5986
segment the <-> segment the = 1.0000
segment the <-> segment all the object = 0.7713
segment the <-> segment everything = 0.8739
segment the <-> panoptic segment = 0.5375
segment all the object <-> segment all the object = 1.0000
segment all the object <-> segment everything = 0.7523
segment all the object <-> panoptic segment = 0.5184
segment everything <-> segment everything = 1.0000
segment everything <-> panoptic segment = 0.5340
panoptic segment <